In [12]:
pip install pytest

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 3.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 824.1 kB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

In [4]:
# Create a Faker instance for generating synthetic data
fake = Faker()

# Function to generate random date ranges for campaigns over the last 12 years
def generate_date_range():
    start_date = fake.date_between(start_date='-12y', end_date='today')
    end_date = start_date + timedelta(days=random.randint(10, 90))  # Campaign duration between 10 to 90 days
    return start_date, end_date

# Generate 100 synthetic campaigns
campaigns_data = []
for campaign_id in range(1, 101):
    campaign_name = fake.catch_phrase()
    start_date, end_date = generate_date_range()
    budget = round(random.uniform(1000, 50000))
    channel = random.choice(["instagram", "reddit", "paid search"])
    target_audience = random.choice(["families", "couples", "seniors"])

    campaigns_data.append({
        "campaign_id": campaign_id,
        "campaign_name": campaign_name,
        "start_date": start_date,
        "end_date": end_date,
        "budget": budget,
        "target_audience": target_audience,
        "channel": channel,
    })

# Create a DataFrame from the synthetic data
campaigns_df = pd.DataFrame(campaigns_data)

# Save the DataFrame to a CSV file
csv_filename = "/Users/ekin/Documents/Projects/mealkit-delivery/data/campaigns.csv"
campaigns_df.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created with 100 synthetic campaigns.")

CSV file '/Users/ekin/Documents/Projects/mealkit-delivery/data/campaigns.csv' has been created with 100 synthetic campaigns.


In [9]:
# Create a Faker instance for generating synthetic data
fake = Faker()

# Load the campaigns data from the CSV file
campaigns_df = pd.read_csv("/Users/ekin/Documents/Projects/mealkit-delivery/data/campaigns.csv")

# Ensure 'start_date' and 'end_date' are datetime objects
campaigns_df['start_date'] = pd.to_datetime(campaigns_df['start_date'])
campaigns_df['end_date'] = pd.to_datetime(campaigns_df['end_date'])

# Number of leads and events
num_leads = 8000
num_clicks = 8000
num_subscribes = 2000

# Define possible values for channels
channels = ["referral", "social media", "paid search"]

# Keep track of leads who clicked but haven't subscribed yet
lead_clicks = {}  # lead_id -> list of campaign_ids
subscribed_leads = set()  # Track leads who have already subscribed

# Generate synthetic events data
events_data = []

# First, generate 8,000 "click" events
for _ in range(num_clicks):
    event_id = fake.uuid4()  # Unique event identifier
    campaign_id = random.randint(1, 100)  # Assuming there are 100 campaigns
    lead_id = random.randint(1, num_leads)  # Assuming there are 8,000 leads

    # Retrieve the campaign's start and end dates
    campaign = campaigns_df[campaigns_df["campaign_id"] == campaign_id].iloc[0]
    start_date = campaign["start_date"]
    end_date = campaign["end_date"]

    # Skip campaigns where the end date is before the start date (just in case)
    if end_date <= start_date:
        continue

    # Generate an event date within the campaign's duration
    event_date = fake.date_time_between_dates(datetime_start=start_date, datetime_end=end_date)

    # Track which campaign the lead clicked in
    if lead_id not in lead_clicks:
        lead_clicks[lead_id] = []
    lead_clicks[lead_id].append(campaign_id)

    # Add the click event to the events_data
    events_data.append({
        "event_id": event_id,
        "campaign_id": campaign_id,
        "lead_id": lead_id,
        "event_type": "click",
        "event_date": event_date,
        "channel": random.choice(channels),
        "subscription_id": None  # No subscription for click events
    })

# Next, generate 2,000 "subscribe" events, ensuring a prior "click" in the same campaign
for _ in range(num_subscribes):
    while True:
        lead_id = random.randint(1, num_leads)  # Pick a random lead

        # Only allow leads who have not subscribed yet
        if lead_id in subscribed_leads:
            continue

        # Ensure the lead has clicked in at least one campaign
        if lead_id in lead_clicks and len(lead_clicks[lead_id]) > 0:
            campaign_id = random.choice(lead_clicks[lead_id])  # Pick a campaign they clicked in
            subscribed_leads.add(lead_id)  # Mark lead as subscribed
            subscription_id = fake.uuid4()  # Generate a unique subscription ID
            break

    # Retrieve the campaign's start and end dates
    campaign = campaigns_df[campaigns_df["campaign_id"] == campaign_id].iloc[0]
    start_date = campaign["start_date"]
    end_date = campaign["end_date"]

    # Generate an event date within the campaign's duration
    event_date = fake.date_time_between_dates(datetime_start=start_date, datetime_end=end_date)

    # Add the subscribe event to the events_data
    events_data.append({
        "event_id": fake.uuid4(),  # Unique event identifier
        "campaign_id": campaign_id,
        "lead_id": lead_id,
        "event_type": "subscribe",
        "event_date": event_date,
        "channel": random.choice(channels),
        "subscription_id": subscription_id  # Assign subscription ID for subscribe events
    })

# Create a DataFrame from the synthetic events data
events_df = pd.DataFrame(events_data)

# Save the DataFrame to a CSV file
csv_filename = "/Users/ekin/Documents/Projects/mealkit-delivery/data/events.csv"
events_df.to_csv(csv_filename, index=False)

print(f"CSV file '{csv_filename}' has been created with {len(events_data)} synthetic events.")

CSV file '/Users/ekin/Documents/Projects/mealkit-delivery/data/events.csv' has been created with 10000 synthetic events.


In [11]:
with open('/Users/ekin/Documents/Projects/mealkit-delivery/code/02_test_gen_data.py', 'w') as f:
    f.write("""
import pytest
import pandas as pd

# Load the generated events CSV file
events_df = pd.read_csv("/Users/ekin/Documents/Projects/mealkit-delivery/data/events.csv")

# Test case: Each lead_id can have a maximum of 1 subscription_id
def test_single_subscription_per_lead():
    leads_with_subscriptions = events_df[events_df['event_type'] == 'subscribe'].groupby('lead_id')['subscription_id'].nunique()
    assert (leads_with_subscriptions <= 1).all(), "Each lead_id should have at most one subscription_id."

# Test case: Each lead_id can have only one "subscribe" event
def test_single_subscribe_event_per_lead():
    leads_with_subscribe_events = events_df[events_df['event_type'] == 'subscribe'].groupby('lead_id')['event_id'].count()
    assert (leads_with_subscribe_events <= 1).all(), "Each lead_id should have at most one 'subscribe' event."

# Test case: Every "subscribe" event is preceded by a "click" event by the same lead_id within the same campaign_id
def test_subscribe_preceded_by_click():
    for _, subscribe_event in events_df[events_df['event_type'] == 'subscribe'].iterrows():
        lead_id = subscribe_event['lead_id']
        campaign_id = subscribe_event['campaign_id']
        # Check if there's a click event by the same lead in the same campaign
        click_event = events_df[
            (events_df['event_type'] == 'click') &
            (events_df['lead_id'] == lead_id) &
            (events_df['campaign_id'] == campaign_id)
        ]
        assert not click_event.empty, f"Subscribe event without preceding click by lead {lead_id} in campaign {campaign_id}"

# Test case: There are exactly 2000 "subscribe" events
def test_subscribe_event_count():
    subscribe_event_count = events_df[events_df['event_type'] == 'subscribe'].shape[0]
    assert subscribe_event_count == 2000, f"Expected 2000 subscribe events, but found {subscribe_event_count}"

# Test case: There are exactly 100 unique campaigns
def test_campaign_count():
    campaign_count = events_df['campaign_id'].nunique()
    assert campaign_count == 100, f"Expected 100 unique campaigns, but found {campaign_count}"
    """)